In [2]:
import socket
import threading
from datetime import datetime
import ssl
import tkinter as tk
from tkinter import scrolledtext, messagebox

# List of vulnerable ports and their names
# You can expand this list according to the services you want to check
vulnerable_ports = {
    20: "FTP (File Transfer Protocol) - Data Transfer",
    21: "FTP (File Transfer Protocol) - Command Control",
    23: "Telnet",
    25: "SMTP (Simple Mail Transfer Protocol)",
    53: "DNS (Domain Name System)",
    110: "POP3 (Post Office Protocol)",
    143: "IMAP (Internet Message Access Protocol)",
    3306: "MySQL Database",
    3389: "Remote Desktop Protocol (RDP)",
    5900: "VNC (Virtual Network Computing)",
}

# Description of possible attacks for each port
attacks_description = {
    20: [
        "Data interception (Packet Sniffing): FTP does not encrypt data, allowing interception.",
        "Command injection: Malicious commands if the configuration is weak."
    ],
    21: [
        "Password brute force: Attempts to guess login credentials.",
        "Exploiting known vulnerabilities: Server-specific FTP vulnerabilities."
    ],
    23: [
        "Data interception: Clear text transmission allows credential interception.",
        "Brute force attacks: Attempt to access via brute force attacks."
    ],
    25: [
        "Email spoofing: Sending fake emails pretending to be a trusted source.",
        "Email interception: Intercepting unencrypted emails."
    ],
    53: [
        "DNS Spoofing: Manipulating DNS responses to redirect to malicious websites.",
        "DDoS Amplification: Using vulnerable DNS servers for DDoS attacks."
    ],
    110: [
        "Email theft: Intercepting unencrypted emails.",
        "Brute force attacks: Attempt to guess passwords to access accounts."
    ],
    143: [
        "Email theft: Intercepting unencrypted emails.",
        "Exploiting IMAP vulnerabilities: Service-specific vulnerabilities."
    ],
    3306: [
        "SQL injection: Exploit unsanitized queries to access sensitive data.",
        "Unauthorized access: Exploits to access the database if unprotected."
    ],
    3389: [
        "Password brute force: Trying to guess credentials for remote control.",
        "Exploiting RDP vulnerabilities: Exploiting RDP-specific vulnerabilities."
    ],
    5900: [
        "Unauthorized access: Attempt to guess passwords for remote desktop.",
        "Exploiting VNC vulnerabilities: Exploit service-specific vulnerabilities."
    ]
}

def scan_port(target, port, result_list):
    """
    Scans a specific port to determine if it is open.

    :param target: Target IP address.
    :param port: Port number to scan.
    :param result_list: List to store scan results.
    """
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    socket.setdefaulttimeout(1)  # Maximum connection timeout (1 second)
    
    try:
        result = sock.connect_ex((target, port))
        if result == 0:  # If the port is open
            port_name = vulnerable_ports.get(port, "Unknown")
            result_list.append(f"Port {port} ({port_name}) is OPEN and VULNERABLE.")
            # Add a description of possible attacks
            attacks = attacks_description.get(port, ["No specific attacks defined for this port."])
            result_list.append("Possible attacks:")
            result_list.extend(f"  - {attack}" for attack in attacks)
    except (socket.error, Exception) as e:
        result_list.append(f"Error scanning port {port}: {e}")
    finally:
        sock.close()  # Close the socket to free resources

def port_scanner(target, result_list):
    """
    Scans vulnerable ports on the target.

    :param target: Target IP address.
    :param result_list: List to store scan results.
    """
    result_list.append(f"\n[*] Scanning {target} for vulnerable ports...")
    result_list.append(f"Scan started at {str(datetime.now())}\n")
    
    threads = []
    for port in vulnerable_ports.keys():
        # Create a thread to scan each port
        thread = threading.Thread(target=scan_port, args=(target, port, result_list))
        threads.append(thread)
        thread.start()
    
    # Wait for all threads to finish
    for thread in threads:
        thread.join()

def check_ssl_certificates(target):
    """
    Verifies the SSL/TLS certificate of the target.

    :param target: Target domain.
    :return: List with the certificate information.
    """
    result_list = []
    try:
        port = 443  # Default port for HTTPS
        context = ssl.create_default_context()
        with socket.create_connection((target, port)) as sock:
            with context.wrap_socket(sock, server_hostname=target) as ssock:
                cert = ssock.getpeercert()
                result_list.append(f"SSL/TLS Certificate for {target}:")
                for key, value in cert.items():
                    result_list.append(f"  {key}: {value}")
    except ssl.SSLError as e:
        result_list.append(f"SSL/TLS certificate error: {e}")
    except Exception as e:
        result_list.append(f"Error verifying SSL/TLS: {e}")
    return result_list

def save_results(results):
    """
    Saves the scan results to a text file.

    :param results: List of scan results.
    """
    try:
        with open("scan_results.txt", "w") as file:
            for result in results:
                file.write(result + "\n")
    except IOError as e:
        messagebox.showerror("File Error", f"Could not save the file: {e}")

def start_scan():
    """
    Starts the scanning process and updates the graphical interface.
    """
    target = domain_entry.get()
    result_list = []
    
    try:
        target_ip = socket.gethostbyname(target)  # Resolve domain to IP
        result_list.append(f"Resolved IP address: {target_ip}")
        
        # Scan vulnerable ports
        port_scanner(target_ip, result_list)
        
        # Verify SSL/TLS certificates
        result_list.extend(check_ssl_certificates(target))
        
        # Save results to a file
        save_results(result_list)
        
        # Display results in the graphical interface text area
        result_area.config(state=tk.NORMAL)
        result_area.delete(1.0, tk.END)
        result_area.insert(tk.END, "\n".join(result_list))
        result_area.config(state=tk.DISABLED)
        
        # Notify the user that the scan is complete
        messagebox.showinfo("Finished", "The scan is complete. Results saved in 'scan_results.txt'.")
    except socket.gaierror:
        messagebox.showerror("Error", "Error resolving IP address. Check the domain or IP.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Graphical interface setup
root = tk.Tk()
root.title("Port Scanner and SSL/TLS Verification")

# Label for domain or IP input
tk.Label(root, text="Enter a website or IP address:").pack(pady=5)

# Entry field for domain or IP
domain_entry = tk.Entry(root, width=50)
domain_entry.pack(pady=5)

# Button to start scanning
scan_button = tk.Button(root, text="Start Scan", command=start_scan)
scan_button.pack(pady=10)

# Text area to display scan results
result_area = scrolledtext.ScrolledText(root, width=80, height=20, wrap=tk.WORD, state=tk.DISABLED)
result_area.pack(pady=10)

# Run the graphical interface
root.mainloop()
